# Problem
Our problem is a classification problem, our target is whether a shot is goal or not.

# Where Can We Use This Model In Real World ?

We can use this model to analyze player performances, understand player habits on the field, determining which players to change in game or find out in advance whether a new player will be compatible with the team or not during transfer seasons.

# Imports

In [608]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import rpy2.robjects.packages as rpackages
from rpy2.robjects.conversion import localconverter
from imblearn.over_sampling import SMOTE
from collections import Counter
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
import h2o
from h2o.automl import H2OAutoML
import pickle

In [609]:
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)

<rpy2.rinterface_lib.sexp.NULLType object at 0x13faad3d0> [0]

# Getting Dataset Over R from Python

In [626]:
pandas2ri.activate()
ro.r('''
        library("worldfootballR")
        laliga <- load_understat_league_shots(league = "La liga")
     ''')
laliga = pandas2ri.rpy2py(ro.r['laliga'])
laliga.drop('league', axis=1, inplace=True)
data = laliga[(laliga['date'] > '2022-01-01') & (laliga['date'] < '2024-06-10')]


→ Data last updated 2024-05-30 18:34:46.012307882309 UTC


We are going to use La Liga dataset between 2023-01-01 and 2024-06-10.

# Data Manipulations and Fixes

We have a problem about NaN values and duplicate features, we are going to fix these problems by manipulating the data.

In [627]:
warnings.filterwarnings('ignore')

def fixDataNaN(df):
    with localconverter(ro.default_converter + pandas2ri.converter):
        df = ro.conversion.py2rpy(df)
pairs = [['x','X'],['y','Y'],['x_g','xG'],['h_a','home_away'],['shot_type','shotType'],['last_action','lastAction']]

def camel_case_columns(df):
    def camel_case(column_name):
        parts = column_name.split('_')
        return str(parts[0] + ''.join(x.title() for x in parts[1:]))
    
    new_columns = []
    for column in df.columns:
        if '_' in column:
            new_columns.append(camel_case(column))
        else:
            new_columns.append(str(column))
    
    df.columns = new_columns
    return df

def fixMergeColumns(dataList, pairs):
    for targetData in dataList:
        for pair in pairs:
            if pair[0] in targetData.columns and pair[1] in targetData.columns:
                targetData['{}'.format(pair[1])].fillna(targetData['{}'.format(pair[0])], inplace=True)
                targetData.drop(columns=['{}'.format(pair[0])], inplace=True)
        targetData = camel_case_columns(targetData)
        fixDataNaN(targetData)

fixMergeColumns([data], pairs)

We are going to train our model to predict whether the position ends up to a goal or not so we need to convert our goal and not goal situations to binary tags.

In [628]:
replacement_dict = {
    'Goal': 'Goal',
    'BlockedShot': 'No Goal',
    'MissedShots': 'No Goal',
    'SavedShot': 'No Goal',
    'ShotOnPost': 'No Goal',
    'OwnGoal': 'No Goal'
}

data['result'] = pd.DataFrame(data['result'].map(replacement_dict))

# Exploring Data

In [613]:
#data.head(2)
print(data['result'].head(5))

66772       Goal
66773    No Goal
66774    No Goal
66775    No Goal
66776    No Goal
Name: result, dtype: object


### Target Variable

Our target variable is "result", this feature represents whether a shot is a goal or not.

### Feature Variables

Our feature variables are going to help our model to learn and predict the target variable.

In [614]:
print(data.drop('result',axis=1).columns.tolist())

['id', 'minute', 'X', 'Y', 'xG', 'player', 'playerId', 'situation', 'season', 'shotType', 'matchId', 'homeTeam', 'awayTeam', 'homeGoals', 'awayGoals', 'date', 'playerAssisted', 'lastAction', 'homeAway']


# Imbalancedness

Our data has imbalancedness problem which might cause our model to learn features of majority target variables better than the minority target variables, it might cause inaccurate predictions.

In [615]:
print(data['result'].value_counts())

result
No Goal    20985
Goal        2379
Name: count, dtype: int64


In [629]:
X = data.drop('result', axis=1)
Y = data['result']
print(Y.head(2))
X = pd.get_dummies(X)

data = pd.concat([pd.DataFrame(Y),pd.DataFrame(X)], axis=1)

66772       Goal
66773    No Goal
Name: result, dtype: object


In [630]:
data_shuffled = data.sample(frac=1, random_state=42)
quarter_length = len(data_shuffled) // 25
df_half_val = data_shuffled.iloc[:quarter_length]
df_half_train = data_shuffled.iloc[quarter_length:]

data = df_half_train
validation_data = df_half_val

We are going to use SMOTE method for oversampling to fix the gap between minority and majority target variables. This method uses clustering methods to create new observations based on original ones.

In [631]:
data['result'] = data['result'].map({'Goal': 1, 'No Goal': 0})
validation_data['result'] = validation_data['result'].map({'Goal': 1, 'No Goal': 0})

In [632]:
Y_validation = validation_data['result']
x_validation = validation_data.drop('result', axis=1)

X = pd.DataFrame(data.drop('result', axis=1))
Y = pd.DataFrame(data['result'])
print(data.result.head(5))


83759    0
67453    1
83379    0
69133    0
89334    0
Name: result, dtype: int64


In [633]:
from numpy import ravel


sm = SMOTE(random_state=42,n_jobs=-1)
x_res, y_res = sm.fit_resample(X, Y)

# concat the resampled data
data_res = pd.concat([pd.DataFrame(y_res), pd.DataFrame(x_res)], axis=1)
print(data_res.result.value_counts())

scaler = MinMaxScaler()

x_res = scaler.fit_transform(x_res)

X_validation = scaler.fit_transform(x_validation)

Y_validation = validation_data['result']

result
0    20143
1    20143
Name: count, dtype: int64


# Model

In [635]:
print(x_res.shape)
print(y_res.shape)

(40286, 2510)
(40286, 1)


In [636]:
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.25, random_state=42, stratify=y_res)

We have found the best model to use is Random Forest after comparing Decision Tree, Logistic Regression,XGBoost and Random Forest.

In [642]:
from sklearn.linear_model import LogisticRegression


model = RandomForestClassifier(random_state=42, n_jobs=-1)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

y_train = y_train.astype(y_pred.dtype)

5-Fold Cross Validation score:

In [643]:
print(cross_val_score(model, x_train, y_train, cv=5, scoring='balanced_accuracy').mean())

0.9561790169015435


### Train - Test Accuracy Comparison to Check Overfitting

Our model seems to have learned train set perfectly, we might have suspected of overfitting much more than the current situation if the test accuracies were bad but accuracy and balanced accuracy scores on train and test sets are very close to each other. Our model is good to go, but we are going to check if hyperparameter tuning or automl could improve our performance.

In [ ]:
test_accuracy = accuracy_score(y_test, y_pred)
test_balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
test_confusion_matrix = confusion_matrix(y_test, y_pred)

train_accuracy = accuracy_score(y_train, model.predict(x_train))
train_balanced_accuracy = balanced_accuracy_score(y_train, model.predict(x_train))
train_confusion_matrix = confusion_matrix(y_train, model.predict(x_train))

In [ ]:
print('Test Accuracy: {}\nTrain Accuracy: {}\n'.format(test_accuracy, train_accuracy))
print('Test Balanced Accuracy: {}\nTrain Balanced Accuracy: {}\n'.format(test_balanced_accuracy, train_balanced_accuracy))
print('Test Confusion Matrix: \n{}\n\nTrain Confusion Matrix: \n{}'.format(test_confusion_matrix, train_confusion_matrix))

Test Accuracy: 0.9475062000551115
Train Accuracy: 0.9588462245085431

Test Balanced Accuracy: 0.9475062000551115
Train Balanced Accuracy: 0.9588462245085431

Test Confusion Matrix: 
[[3528  101]
 [ 280 3349]]

Train Confusion Matrix: 
[[10734   152]
 [  744 10142]]


### Hyperparameter Tuning

We are going to use the Halving Random Search method and 5-Fold Cross Validation together to find the best hyperparameters, evaluating our iterations by balanced accuracy scores.

The Halving Random Search method uses an elimination system as its base idea: the best of the two compared hyperparameter sets rises above on the leaderboard, and we get the best hyperparameters after the final round. Another pro of Halving Random Search is that it is much faster than Random Search.

In [649]:
param_grid = {
    'n_estimators': np.linspace(10, 1000, num=1000, dtype=int),  # Ağaç sayısı
    'max_depth': np.linspace(10, 100, num=50, dtype=int),          # Ağaçların maksimum derinliği
    'min_samples_split': [2, 5, 10],              # Bir düğümü bölmek için gereken minimum örnek sayısı
    'min_samples_leaf': [1, 2, 4],                # Bir yaprak düğümde olması gereken minimum örnek sayısı
    'max_features': ['auto', 'sqrt'],             # Her bir bölme için maksimum özellik sayısı
    'bootstrap': [True, False]                    # Bootstrap örneklemesinin kullanılması
}


halving = HalvingRandomSearchCV(model, param_grid, factor=3, resource='n_samples', max_resources=5000, random_state=42, verbose=3,scoring='balanced_accuracy', n_jobs=-1)
halving.fit(x_train, ravel(y_train))
print("Best Params:{}/nBest Balanced Accuracy:{}".format(halving.best_params_,halving.best_score_))

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 20
max_resources_: 5000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 250
n_resources: 20
Fitting 5 folds for each of 250 candidates, totalling 1250 fits
[CV 3/5] END bootstrap=True, max_depth=77, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=163;, score=(train=nan, test=nan) total time=   0.0s
[CV 1/5] END bootstrap=True, max_depth=77, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=163;, score=(train=nan, test=nan) total time=   0.0s
[CV 2/5] END bootstrap=True, max_depth=77, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=163;, score=(train=nan, test=nan) total time=   0.0s
[CV 4/5] END bootstrap=True, max_depth=77, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=163;, score=(train=nan, test=nan) total time=   0.0s
[CV 5/5] END bootstrap=True, max_depth=77, max_features=a

We did not get what we wanted from hyperparameter tuning, our balanced accuracy decreased and it might not be ideal to consume more time to random searching, so let's see how AutoML is going to work.

In [ ]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,7 hours 21 mins
H2O_cluster_timezone:,Europe/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.2
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_sezaiufukoral_x1b2rc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,137.7 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [ ]:
# hf = h2o.H2OFrame(pd.concat([pd.DataFrame(x_res), pd.DataFrame(y_res)], axis=1))

# train_hf, test_hf = hf.split_frame(ratios=[0.75], seed = 1)

### AutoML

We are going to use the H2O library because of its ease of use. We are going to set the maximum runtime to 300 seconds so our process won't consume too much time. AutoML is going to use 5-Fold Cross Validation by default and compare balanced accuracies and more metrics to choose the best model for our data.

In [ ]:
# aml = H2OAutoML(max_models = 20,
#                 balance_classes=True,
# 		seed =1,max_runtime_secs=600,verbosity='none')

# aml.train(training_frame = train_hf, y = 'result')

AutoML found the best model to be Gradient Boosting Machines. This algorithm is based on the idea of creating an ensemble of weak learners, typically decision trees, in a sequential manner. Each new model attempts to correct the errors of the previous models.

Our new model's performance is almost the same as vanilla Random Forest. However, AutoML checked more metrics to validate that this is the right model for our data. Both models are black box models, so it does not affect us which one we choose for the sake of interpretability. We can trust more in the one that AutoML found for its generalizability because it checked more metrics when building the model. So, we are good to go now.

In [ ]:
# lb = aml.leaderboard
# lb.head(rows=lb.nrows)

In [ ]:
# warnings.filterwarnings('ignore')
# preds = aml.leader.predict(test_hf)

# y_test_gbm = test_hf['result'].as_data_frame().values.flatten()
# y_pred_gbm = preds['predict'].as_data_frame().values.flatten()

# balanced_acc = balanced_accuracy_score(y_test_gbm, y_pred_gbm)
# print("Balanced Accuracy Score: ", balanced_acc)

# Validation of the Model

We are going to test our model with a new, unseen data.

In [ ]:
# warnings.filterwarnings('ignore')

# validation_x_hf = h2o.H2OFrame(pd.DataFrame(X_validation))
# validation_y_hf = h2o.H2OFrame(pd.DataFrame(Y_validation))

                               
# preds_validation = aml.leader.predict(validation_x_hf)

# y_validation_test_gbm = validation_y_hf['result'].as_data_frame().values.flatten()
# y_validation_pred_gbm = preds_validation['predict'].as_data_frame().values.flatten()

# balanced_acc = balanced_accuracy_score(y_validation_test_gbm, y_validation_pred_gbm)
# print("Balanced Accuracy Score: ", balanced_acc)

# print(confusion_matrix(y_validation_test_gbm, y_validation_pred_gbm))

In [651]:
y_validation_pred = model.predict(X_validation)

print(confusion_matrix(Y_validation, y_validation_pred))

print(balanced_accuracy_score(Y_validation, y_validation_pred))

[[806  36]
 [ 62  30]]
0.6416658060518434


In [650]:
halving.best_estimator_.fit(X_validation, Y_validation)
print(confusion_matrix(Y_validation, halving.best_estimator_.predict(X_validation)))
print(balanced_accuracy_score(Y_validation, halving.best_estimator_.predict(X_validation)))

[[842   0]
 [  6  86]]
0.9673913043478262
